In [33]:
import PyPDF2
import os
import re
import openai
import numpy as np
import pandas as pd
from joblib import load
from openai import OpenAI

In [34]:
def extract_tables_and_pages(pdf_path):
    """Extract table titles from the paper"""
    
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        data = []  # This will store tuples of (pdf_path, title, page)

        # Regular expression to find table titles
        table_title_regex = re.compile(r'[Tt][Aa][Bb][Ll][Ee]+\.?+\s*[A-Z]?\d+\.?\s+[A-Z]+.*')

        for i in range(num_pages):
            page = reader.pages[i]
            text = page.extract_text()
            if text:
                found_titles = table_title_regex.findall(text)
                for title in found_titles:
                    data.append((pdf_path, title, i + 1))  # Append the path, title, and page number

    # Create a DataFrame from the list of tuples
    df = pd.DataFrame(data, columns=['pdf_path', 'Table_title', 'Page'])
    df['Table_title'] = df['Table_title'].str.replace(r'\n', ' ', regex=True)
    return df

In [35]:
def extract_file_name_from_path(file_path):
    """Extract the file name"""
    
    file_name_with_extension = file_path.split('/')[-1]
    # Remove the file extension
    title = file_name_with_extension.replace('.pdf', '')
    return title

In [36]:
def generate_embeddings(text):
    """Initial the embedding model"""
    
    openai_api_key= os.environ.get("OPENAI_API_KEY_SBR")
    
    client = OpenAI(api_key=openai_api_key)
    
    # make api call
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    
    # return text embedding
    return response.data

In [37]:
def df_to_csv(df, file_path):
    """Write a DataFrame to a CSV file."""
    df.to_csv(file_path, index=False, escapechar='\\')

**1. Extract table titles from PDFs**

In [38]:
#read all pdf files from a specific directory
directory = 'C:/Users/89751/OneDrive/desktop/Document/'
files = os.listdir(directory)
pdf_files = [os.path.join(directory, file) for file in files] 
pdf_files

['C:/Users/89751/OneDrive/desktop/Document/1.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/10.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/11.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/12.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/13.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/14.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/15.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/16.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/17.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/18.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/19.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/2.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/20.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/21.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/22.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/23.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/24.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/25.pdf',
 'C:/Users/89751/OneDrive/desktop/Document/26.pdf',
 'C:/Users/897

In [41]:
df1 = [extract_tables_and_pages(path) for path in pdf_files]
df1 = pd.concat(df1, ignore_index=True)
df1

,pdf_path,Table_title,Page
0,C:/Users/89751/OneDrive/desktop/Document/1.pdf,Table 1 The amount of CBM drainage and utiliza...,5
1,C:/Users/89751/OneDrive/desktop/Document/1.pdf,Table 3 Transport mode and fuel mix [29].,6
2,C:/Users/89751/OneDrive/desktop/Document/1.pdf,Table 2 Loss rate in coal washing and selectio...,6
3,C:/Users/89751/OneDrive/desktop/Document/1.pdf,Table 7 Emission and mitigation by building ma...,7
4,C:/Users/89751/OneDrive/desktop/Document/1.pdf,"Table 8 Results of CO 2,eof each process in th...",7
...,...,...,...
169,C:/Users/89751/OneDrive/desktop/Document/8.pdf,Table 6 LCA endpoint results of energy source ...,10
170,C:/Users/89751/OneDrive/desktop/Document/9.pdf,table 1. CO 2from biogas and ﬂue gases (cement...,3
171,C:/Users/89751/OneDrive/desktop/Document/9.pdf,Table 1 Potential CO 2sources in Germany,4
172,C:/Users/89751/OneDrive/desktop/Document/9.pdf,table 2. Biogas and cement production itself a...,4


In [42]:
#replace pdf_path with the file_name
df1['pdf_path'] = df1['pdf_path'].apply(extract_title_from_path)
df1 = df.rename(columns={'pdf_path': 'file_name'})
df1.head()

,file_name,Table_title,Page
0,1,Table 1 The amount of CBM drainage and utiliza...,5
1,1,Table 3 Transport mode and fuel mix [29].,6
2,1,Table 2 Loss rate in coal washing and selectio...,6
3,1,Table 7 Emission and mitigation by building ma...,7
4,1,"Table 8 Results of CO 2,eof each process in th...",7


**2. Table title embedding and classification**

In [43]:
#generate embeddings for table title
text_embedding_list = generate_embeddings(df1['Table_title'])
text_embedding_list = [text_embedding_list[i].embedding for i in range(len(text_embedding_list))]

In [44]:
#load the pre-trained random forest model
clf_loaded = load('random_forest_model.pkl')

In [53]:
embeddings_array = np.array(text_embedding_list)

# Make predictions
predictions = clf_loaded.predict(embeddings_array)
predictions

C:\Software\Anaconda\envs\LangChain\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False,  True,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False,  True,  True, False, False, False, False,
        True,  True, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False,

In [46]:
df2 = pd.DataFrame(predictions)
df2

,0
0,False
1,False
2,False
3,False
4,False
...,...
169,False
170,False
171,False
172,False


In [61]:
#combine data
combined_df = pd.concat([df1, df2], axis=1).fillna(False)
combined_df.rename(columns={0: 'is LCI inventory table?'}, inplace=True)
combined_df.head()

C:\Users\89751\AppData\Local\Temp\ipykernel_944\3400081224.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df = pd.concat([df1, df2], axis=1).fillna(False)


,file_name,Table_title,Page,is LCI inventory table?
0,1,Table 1 The amount of CBM drainage and utiliza...,5,False
1,1,Table 3 Transport mode and fuel mix [29].,6,False
2,1,Table 2 Loss rate in coal washing and selectio...,6,False
3,1,Table 7 Emission and mitigation by building ma...,7,False
4,1,"Table 8 Results of CO 2,eof each process in th...",7,False


In [ ]:
#export the data
file_name = 'result.csv'
df_to_csv(df, file_name)